# Scrape Data From Monster For All States

In [1]:
import pandas as pd # For Data Storage
import requests # For website connection
from bs4 import BeautifulSoup # For HTML parsing
import time # For sleep 
import re # Regular expressions for removing non-ascii terms
from itertools import groupby # For removing duplicates from lists
import math # Need ceiling expression
from nltk.corpus import stopwords # For filtering out words like 'is', 'the', 'of'
stop_words = set(stopwords.words("english")) #initialize stopwords    
import matplotlib.pyplot as plt
import random
import datetime

# Initialize Empty Lists (For Dictionary)

In [2]:
#Initialize Empty State Set
state = []
job_title = []

#Initialize Empty Skills List
r = []
python = []
java = []
cpp = []
csharp = [] 
c = [] 
ruby = []
perl = []
sas = [] 
spss = []
matlab = []
javascript = []
scala = []
excel = []
vba = []
tableau = []
d3 = []
hadoop = []
mapreduce = []
spark = []
pig = []
hive = []
shark = []
oozie = []
zookeeper = []
flume = []
mahout = []
sql = [] 
mysql = []
tsql = []
oracle = []
postgresql = []
nosql =[]
sybase = []
rethinkdb = []
memcacheddb = []
couchdb = []
hbase = []
cassandra = []
mongodb = []
db2 = []
git = []
tensorflow = []
keras = []

skills = ['r',
'python',
'java',
'c++',
'c#',
'c',
'ruby',
'perl',
'sas',
'spss',
'matlab',
'javascript',
'scala', 
'excel',
'vba',
'tableau',
'd3',
'hadoop',
'mapreduce',
'spark',
'pig',
'hive',
'shark',
'oozie',
'zookeeper',
'flume',
'mahout',
'sql',
'tsql',
'mysql',
'oracle',
'postgresql',
'nosql',
'sybase',
'rethinkdb',
'memcacheddb',
'couchdb',
'hbase',
'cassandra',
'mongodb',
'db2',
'git',
'tensorflow',
'keras'
]

# Initialize Dictionary

In [3]:
dictionary = {
    'job_title':job_title,
    'state':state,
    'r':r,
    'python':python,
    'java':java,
    'c++':cpp,
    'c#':csharp,
    'c':c,
    'ruby':ruby,
    'perl':perl,
    'sas':sas,
    'spss':spss,
    'matlab':matlab,
    'javascript':javascript,
    'scala':scala,
    'excel':excel,
    'vba':vba,
    'tableau':tableau,
    'd3':d3,
    'hadoop':hadoop,
    'mapreduce':mapreduce,
    'spark':spark,
    'pig':pig,
    'hive':hive,
    'shark':shark,
    'oozie':oozie,
    'zookeeper':zookeeper,
    'flume':flume,
    'mahout':mahout,
    'sql':sql, 
    'tsql':tsql,
    'mysql':mysql,
    'oracle':oracle,
    'postgresql':postgresql,
    'nosql':nosql,
    'sybase':sybase,
    'rethinkdb':rethinkdb,
    'memcacheddb':memcacheddb,
    'couchdb':couchdb,
    'hbase':hbase,
    'cassandra':cassandra,
    'mongodb':mongodb,
    'db2':db2,
    'git':git,
    'tensorflow':tensorflow,
    'keras':keras}

# Define Func: Add A Row of Data To Dictionary (For a Job Description)

In [4]:
def addValsToDict(url_state,url_job_title):
    #Append job_title
    dictionary['job_title'].append(url_job_title)
    
    #Append state
    dictionary['state'].append(url_state)

    #Check if the each skill is in the wrangled text
    for skill in skills:
        if (skill in text_final) == True:
            dictionary[skill].append(1)
        else:
            dictionary[skill].append(0)

# Define States List

In [5]:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

## import proxys

#Get top 10 proxies from http://free-proxy.cz/en/proxylist/country/US/https/ping/level1
https_proxys = ['34.192.220.22:808',
              '206.189.112.106:3128',
              '209.97.191.169:3128',
              '157.230.34.190:1111',
              '157.230.45.121:1111',
              '23.108.64.65:8118',
              '157.230.33.37:1111',
              '52.128.60.130:50692',
              '12.218.209.130:53281',
              '38.134.10.106:53281'
               ]

In [6]:
# Get proxies from filter
https_proxys = list(pd.read_csv('filtered_https_proxys.txt',header=None)[0])

## import headers

In [7]:
user_agents = pd.read_csv('headers.txt',delimiter = '\t',header=None).T[0]

In [8]:
user_agents = user_agents[0:-1] # Get rid of last element (it is nan)

In [9]:
user_agents = list(user_agents)

# Create select a random proxy and header funcitons

In [10]:
def randIndex(lengthList):
    index = random.randint(0,lengthList-1)
    return index

def randProxy():
    index = randIndex(len(https_proxys))
    return https_proxys[index]
    
def randHeader():
    index = randIndex(len(user_agents))
    return user_agents[index]

# Initialize Headers Dictionary

In [11]:
header = {'User-Agent':randHeader()}

# Initialize Proxy Dictionary

In [12]:
proxy = {'https':randProxy()}

## For Each State:

- ## Get the urls that lists up to 250 job openings

In [161]:
count = 0

# Initialize Empty URL Dictionary
final_state_url_dict = {} 

while len(final_state_url_dict.keys()) < 50:
    for url_state in states:
        try:
            if url_state not in final_state_url_dict.keys():
                time.sleep(1) # sleep 1 second

                # Assign random proxy
                proxy['https'] = randProxy()

                # Assign random header
                header['User-Agent'] = randHeader()

                count += 1
                print("Number of states left to scrape: "+str(50-count))

                url = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1'        

                response = requests.get(url, proxies=proxy, headers=header, timeout=5) #Try to get the html code

                soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

                time.sleep(1) #pause code for 1 sec

                #Get the total number of listed jobs 
                totalJobOpenings = float(re.sub('[^0-9]','',soup.html.body.find_all('h2')[0].text.strip()))

                #Get the total number of pages (up to 10) and a max of 25 per page
                totalPages = int(min(math.ceil(totalJobOpenings/25),10))

                #Store the Final URL in a dictionary each state
                final_state_url_dict[url_state] = 'https://www.monster.com/jobs/search/?q=data-scientist&where='+url_state+'&stpage=1&page='+str(totalPages)


        except Exception,e: 
            count -= 1
            print str(e)

Number of states left to scrape: 49
Number of states left to scrape: 48
Number of states left to scrape: 47
Number of states left to scrape: 46
Number of states left to scrape: 45
Number of states left to scrape: 44
Number of states left to scrape: 43
Number of states left to scrape: 42
Number of states left to scrape: 41
HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=data-scientist&where=FL&stpage=1 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1e8f3250>, 'Connection to 40.114.109.214 timed out. (connect timeout=5)'))
Number of states left to scrape: 41
Number of states left to scrape: 40
Number of states left to scrape: 39
Number of states left to scrape: 38
Number of states left to scrape: 37
Number of states left to scrape: 36
Number of states left to scrape: 35
('Connection aborted.', BadStatusLine("''",))
Number of states left to scrape: 35
Number of states left to scrape: 34
Number 

- ## Get the list of job post URLs -> For each URL: check if each skill exists and store data in dictionary

In [ ]:


# End the loop when all job state urls have been processed
while len(final_state_url_dict.keys()) > 0:
    for state in final_state_url_dict.keys():
        try:
            print("Number of states left to scrape: "+str(len(final_state_url_dict.keys())))
            time.sleep(1) # sleep 1 second

            # Assign random proxy
            proxy['https'] = randProxy()

            # Assign random header
            header['User-Agent'] = randHeader()
            
            #Access the job listing page for a state
            response = requests.get(final_state_url_dict[state],proxies=proxy,headers=header,timeout=3) #Get the html code
            soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

            links = []
            job_titles = {}
            # Access every URL within the page and store only urls for job openings
            for link in soup.find_all('a'):
                if link.get('href')[0:21] == 'https://job-openings.': ## a job opening always begins with 'https://job-openings.monster.com'
                    links.append(link.get('href'))
                    job_titles[link.get('href')] = link.contents[0].strip('\n').strip('\r')



            # Access each job posting and store it's html code into a beautiful soup object
            # End the loop when all job title urls have been processed
            while len(job_titles.keys()) > 0: 
                for url in job_titles.keys():
                    try:
                        # Assign random proxy
                        proxy['https'] = randProxy()

                        # Assign random header
                        header['User-Agent'] = randHeader()

                        count2 += 1
                        print("Number of jobs left to scrape: "+str(len(job_titles.keys())))
                        
                        response = requests.get(url,proxies=proxy,headers=header,timeout=3) #Get the html code
                        soup = BeautifulSoup(response.text, "html.parser") #Store html in a soup object 

                        #Get and clean the text from the job posting
                        text = soup.body.find_all('script')[1]

                        # Convert to string -> Convert to lower case 
                        text_wrangled = str(text).lower()
                        # replace (all non ascii characters,'.' ,'+' , and '3') with ','
                        text_wrangled = re.sub("[^a-zA-Z.+#3]",',',text_wrangled)
                        # split string into a list separated by ','
                        text_wrangled = text_wrangled.split(',')
                        # remove all consecutive duplicates 
                        text_wrangled = groupby(text_wrangled)
                        text_wrangled = [x[0] for x in text_wrangled]
                        # Remove the stopwords and empty spaces
                        text_final = [x for x in text_wrangled if not x in stop_words and not x =='']
                        #Add the values to the Dictionary
                        addValsToDict(state,job_titles[url])
                        
                        #delete the job post link finished
                        del job_titles[url]
                        
                    except Exception,e: 
                        print str(e)
                        
            #delete state when finished
            del final_state_url_dict[state]
       
        except Exception,e: 
            print str(e)
            
            

Number of states left to scrape: 50
HTTPSConnectionPool(host='www.monster.com', port=443): Max retries exceeded with url: /jobs/search/?q=data-scientist&where=WA&stpage=1&page=10 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1f10f790>, 'Connection to 217.69.10.117 timed out. (connect timeout=3)'))
Number of states left to scrape: 50
Number of jobs left to scrape: 100
Number of jobs left to scrape: 99
Number of jobs left to scrape: 98
Number of jobs left to scrape: 97
Number of jobs left to scrape: 96
Number of jobs left to scrape: 95
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /director-of-business-analytics-americas-emea-wilmington-de-us-220-life-ins-co-of-north-america/d7bfcb8f-e446-4e17-848a-71797fc23e53 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1d8d7650>, 'Connection to 167.99.7.198 timed out. (connect timeout=3)'))
Number of jobs left to scra

Number of jobs left to scrape: 10
Number of jobs left to scrape: 9
Number of jobs left to scrape: 8
Number of jobs left to scrape: 7
Number of jobs left to scrape: 6
Number of jobs left to scrape: 5
list index out of range
Number of jobs left to scrape: 5
Number of jobs left to scrape: 4
Number of jobs left to scrape: 3
('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 3
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /model-ownership-analyst-180122918-wilmington-de-us-jp-morgan/ade0ec4c-f513-44b7-a950-65354788ca3e (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1e0f5950>, 'Connection to 64.91.248.243 timed out. (connect timeout=3)'))
Number of jobs left to scrape: 3
Number of jobs left to scrape: 2
Number of jobs left to scrape: 1
Number of states left to scrape: 49
Number of jobs left to scrape: 157
Number of jobs left to scrape: 156
Number of jobs left to scrape: 155
Nu

Number of jobs left to scrape: 40
Number of jobs left to scrape: 39
Number of jobs left to scrape: 38
Number of jobs left to scrape: 37
Number of jobs left to scrape: 36
Number of jobs left to scrape: 35
Number of jobs left to scrape: 34
('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 34
Number of jobs left to scrape: 33
Number of jobs left to scrape: 32
Number of jobs left to scrape: 31
Number of jobs left to scrape: 30
Number of jobs left to scrape: 29
list index out of range
Number of jobs left to scrape: 29
Number of jobs left to scrape: 28
Number of jobs left to scrape: 27
list index out of range
Number of jobs left to scrape: 27
Number of jobs left to scrape: 26
Number of jobs left to scrape: 25
Number of jobs left to scrape: 24
Number of jobs left to scrape: 23
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /analyst-kohler-wi-us-kohler-company/84f24832-2820-4d99-b384-34e8cd90beef (Caused by ConnectTimeo

HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /on-demand-life-insurance-agent-work-when-where-you-want-green-river-wy-us-assurance/e84cde66-537c-4fc7-8fef-257e489b3f9f (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of jobs left to scrape: 45
Number of jobs left to scrape: 44
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /remote-life-insurance-agent-100-commission-free-leads-no-green-river-wy-us-assurance/916bdbd5-3951-42a1-a7de-983b976d98c6 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of jobs left to scrape: 44
Number of jobs left to scrape: 43
Number of jobs left to scrape: 42
Number of jobs left to scrape: 41
Number of jobs left to scrape: 40
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /contract-life-insurance-agent-100-commission-autonomy-f-cheyenne-wy-us-assurance/68

HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /computer-science-research-institute-postdoc-albuquerque-nm-us-sandia-labs/604bc9c6-d4af-4e59-8e53-d54e3dc3d6a1 (Caused by ProxyError('Cannot connect to proxy.', timeout('timed out',)))
Number of jobs left to scrape: 25
Number of jobs left to scrape: 24
Number of jobs left to scrape: 23
Number of jobs left to scrape: 22
Number of jobs left to scrape: 21
('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 21
Number of jobs left to scrape: 20
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /remote-sensing-data-scientist-early-mid-career-albuquerque-nm-us-sandia-labs/d06ed0ab-d914-491b-88ed-98ad61e4ed25 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1e345210>, 'Connection to 40.114.109.214 timed out. (connect timeout=3)'))
Number of jobs left to scrape: 20
Number of jobs left to scra

...

Number of jobs left to scrape: 58
Number of jobs left to scrape: 57
Number of jobs left to scrape: 56
Number of jobs left to scrape: 55
Number of jobs left to scrape: 54
Number of jobs left to scrape: 53
Number of jobs left to scrape: 52
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /remote-life-insurance-agent-100-commission-free-leads-no-metairie-la-us-assurance/d2858b63-c6e6-461d-9996-ae1b81769bc6 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1e215450>, 'Connection to 64.91.248.243 timed out. (connect timeout=3)'))
Number of jobs left to scrape: 52
Number of jobs left to scrape: 51
Number of jobs left to scrape: 50
('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 50
Number of jobs left to scrape: 49
Number of jobs left to scrape: 48
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /life-insurance-agent-remote-no-marketi

Number of jobs left to scrape: 203
Number of jobs left to scrape: 202
Number of jobs left to scrape: 201
Number of jobs left to scrape: 200
Number of jobs left to scrape: 199
Number of jobs left to scrape: 198
Number of jobs left to scrape: 197
Number of jobs left to scrape: 196
Number of jobs left to scrape: 195
Number of jobs left to scrape: 194
Number of jobs left to scrape: 193
HTTPSConnectionPool(host='job-openings.monster.com', port=443): Max retries exceeded with url: /contract-life-insurance-agent-100-commission-autonomy-f-morristown-tn-us-assurance/5a048e47-7d1a-4a03-96ff-3afd419b78a6 (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1a1e105950>: Failed to establish a new connection: [Errno 61] Connection refused',)))
Number of jobs left to scrape: 193
Number of jobs left to scrape: 192
Number of jobs left to scrape: 191
Number of jobs left to scrape: 190
Number of jobs left to scrape: 189
Number of jo

Number of jobs left to scrape: 105
list index out of range
Number of jobs left to scrape: 105
Number of jobs left to scrape: 104
Number of jobs left to scrape: 103
Number of jobs left to scrape: 102
Number of jobs left to scrape: 101
Number of jobs left to scrape: 100
Number of jobs left to scrape: 99
Number of jobs left to scrape: 98
Number of jobs left to scrape: 97
Number of jobs left to scrape: 96
Number of jobs left to scrape: 95
Number of jobs left to scrape: 94
Number of jobs left to scrape: 93
Number of jobs left to scrape: 92
Number of jobs left to scrape: 91
Number of jobs left to scrape: 90
Number of jobs left to scrape: 89
Number of jobs left to scrape: 88
Number of jobs left to scrape: 87
Number of jobs left to scrape: 86
Number of jobs left to scrape: 85
Number of jobs left to scrape: 84
Number of jobs left to scrape: 83
Number of jobs left to scrape: 82
Number of jobs left to scrape: 81
Number of jobs left to scrape: 80
Number of jobs left to scrape: 79
('Connection abor

list index out of range
Number of jobs left to scrape: 208
Number of jobs left to scrape: 207
Number of jobs left to scrape: 206
Number of jobs left to scrape: 205
Number of jobs left to scrape: 204
Number of jobs left to scrape: 203
Number of jobs left to scrape: 202
('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 202
Number of jobs left to scrape: 201
Number of jobs left to scrape: 200
Number of jobs left to scrape: 199
Number of jobs left to scrape: 198
Number of jobs left to scrape: 197
('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 197
list index out of range
Number of jobs left to scrape: 197
list index out of range
Number of jobs left to scrape: 197
Number of jobs left to scrape: 196
Number of jobs left to scrape: 195
Number of jobs left to scrape: 194
Number of jobs left to scrape: 193
Number of jobs left to scrape: 192
Number of jobs left to scrape: 191
Number of jobs left to scrape: 190
Number of jobs left to scrape: 

Number of jobs left to scrape: 34
Number of jobs left to scrape: 33
Number of jobs left to scrape: 32
Number of jobs left to scrape: 31
Number of jobs left to scrape: 30
Number of jobs left to scrape: 29
Number of jobs left to scrape: 28
Number of jobs left to scrape: 27
Number of jobs left to scrape: 26
Number of jobs left to scrape: 25
Number of jobs left to scrape: 24
Number of jobs left to scrape: 23
Number of jobs left to scrape: 22
Number of jobs left to scrape: 21
Number of jobs left to scrape: 20
Number of jobs left to scrape: 19
Number of jobs left to scrape: 18
Number of jobs left to scrape: 17
Number of jobs left to scrape: 16
Number of jobs left to scrape: 15
Number of jobs left to scrape: 14
Number of jobs left to scrape: 13
('Connection aborted.', BadStatusLine("''",))
Number of jobs left to scrape: 13
Number of jobs left to scrape: 12
Number of jobs left to scrape: 11
Number of jobs left to scrape: 10
Number of jobs left to scrape: 9
Number of jobs left to scrape: 8
Numb

# Make the DataFrame

In [92]:
#for x in dictionary:
#    print(len(dictionary[str(x)]))
data = pd.DataFrame(dictionary)

# Store to File

In [45]:
now = datetime.datetime.now()
date = str(now.month)+'-'+str(now.day)+'-'+str(now.year)
data.to_csv('scraped_data+'+date+'.txt')

199

In [ ]:
for link in soup.find_all('a'):
                if link.get('href')[0:21] == 'https://job-openings.': ## a job opening always begins with 'https://job-openings.monster.com'
                    links.append(link.get('href'))
                    job_titles[link.get('href')] = link.contents[0].strip('\n').strip('\r')

In [2]:
url = 'https://www.proxy-list.download/HTTPS'

In [18]:
req = requests.get(url)
print(req)

<Response [200]>


In [16]:
html = req.text

In [17]:
html

u'<!DOCTYPE html>\n<html lang="en">\n\n<head>\n\t<!-- Global site tag (gtag.js) - Google Analytics -->\n\t<script async src="https://www.googletagmanager.com/gtag/js?id=UA-118201503-1"></script>\n\t<script>\n\t\twindow.dataLayer = window.dataLayer || [];\n\t\tfunction gtag() { dataLayer.push(arguments); }\n\t\tgtag(\'js\', new Date());\n\n\t\tgtag(\'config\', \'UA-118201503-1\');\n\t</script>\n\t<script>var proxytype = \'https\';</script>\n\t<meta charset="UTF-8">\n\t<meta name="description" content="Download IP:PORT SSL / HTTPS proxy list txt checked and sorted by country and anonymity level.">\n\t<meta name="keywords" content="SSL proxies, Free SSL Proxy list, proxy list txt, proxy list daily">\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0" />\n\t<title>Free SSL / HTTPS Proxy List .TXT</title>\n\t<link rel="alternate" hreflang="es" href="https://www.proxy-list.download/es/HTTPS" />\n\n\t<link rel="stylesheet" href="https://www.proxy-list.download/css/estilos